In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
train_data

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0
1,1,2013-01-01,1,BABY CARE,0.000,0
2,2,2013-01-01,1,BEAUTY,0.000,0
3,3,2013-01-01,1,BEVERAGES,0.000,0
4,4,2013-01-01,1,BOOKS,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8


In [3]:
# Cuenta la frecuencia de cada valor en la columna 'type'
value_counts = train_data["onpromotion"].value_counts()

# Cuenta los valores NaN en la columna 'type'
nan_count = train_data["onpromotion"].isna().sum()

# Agrega el recuento de NaN al resultado anterior
value_counts_with_nan = pd.concat([value_counts, pd.Series([nan_count], index=['NaN'])])

print(value_counts_with_nan)

0      2389559
1       174551
2        79386
3        45862
4        31659
        ...   
452          1
642          1
305          1
425          1
NaN          0
Length: 363, dtype: int64


In [4]:
print(train_data["store_nbr"].unique())

[ 1 10 11 12 13 14 15 16 17 18 19  2 20 21 22 23 24 25 26 27 28 29  3 30
 31 32 33 34 35 36 37 38 39  4 40 41 42 43 44 45 46 47 48 49  5 50 51 52
 53 54  6  7  8  9]


In [5]:
store_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
store_data

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4
5,6,Quito,Pichincha,D,13
6,7,Quito,Pichincha,D,8
7,8,Quito,Pichincha,D,8
8,9,Quito,Pichincha,B,6
9,10,Quito,Pichincha,C,15


Ahora voy a mezclar los datos de los distintos csv para poder tener un dataset con todos los datos completos

# Stores

In [6]:
train_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
store_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')

merged_df = pd.merge(train_data, store_data, on='store_nbr', how='left')

merged_df.to_csv('data_with_stores', index=False)

In [7]:
merged_df

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6


In [8]:
# Cuenta la frecuencia de cada valor en la columna 'type'
value_counts = merged_df["city"].value_counts()

# Cuenta los valores NaN en la columna 'type'
nan_count = merged_df["city"].isna().sum()

# Agrega el recuento de NaN al resultado anterior
value_counts_with_nan = pd.concat([value_counts, pd.Series([nan_count], index=['NaN'])])

print(value_counts_with_nan)

Quito            1000296
Guayaquil         444576
Santo Domingo     166716
Cuenca            166716
Latacunga         111144
Manta             111144
Machala           111144
Ambato            111144
Quevedo            55572
Esmeraldas         55572
Loja               55572
Libertad           55572
Playas             55572
Daule              55572
Babahoyo           55572
Cayambe            55572
Salinas            55572
Puyo               55572
Guaranda           55572
Ibarra             55572
Riobamba           55572
El Carmen          55572
NaN                    0
dtype: int64


# Holidays

In [9]:
holidays_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
holidays_data

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [10]:
merged_df_2 = pd.merge(merged_df, holidays_data, on='date', how='left')
merged_df_2

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [11]:
print(merged_df_2.isna().sum())

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type_x               0
cluster              0
type_y         2551824
locale         2551824
locale_name    2551824
description    2551824
transferred    2551824
dtype: int64


In [12]:
# Cuenta la frecuencia de cada valor en la columna 'type'
value_counts = merged_df_2["type_y"].value_counts()

# Cuenta los valores NaN en la columna 'type'
nan_count = merged_df_2["type_y"].isna().sum()

# Agrega el recuento de NaN al resultado anterior
value_counts_with_nan = pd.concat([value_counts, pd.Series([nan_count], index=['NaN'])])

print(value_counts_with_nan)


Holiday        304722
Event           99792
Additional      67716
Transfer        16038
Work Day         8910
Bridge           5346
NaN           2551824
dtype: int64


In [13]:
# Filtra las filas donde 'category' tiene un valor NaN
nan_rows = merged_df_2[merged_df_2['type_y'].isna()]

# Muestra los primeros ejemplos (por ejemplo, los primeros 5)
print(nan_rows.head())


        id        date  store_nbr      family   sales  onpromotion   city  \
1782  1782  2013-01-02          1  AUTOMOTIVE     2.0            0  Quito   
1783  1783  2013-01-02          1   BABY CARE     0.0            0  Quito   
1784  1784  2013-01-02          1      BEAUTY     2.0            0  Quito   
1785  1785  2013-01-02          1   BEVERAGES  1091.0            0  Quito   
1786  1786  2013-01-02          1       BOOKS     0.0            0  Quito   

          state type_x  cluster type_y locale locale_name description  \
1782  Pichincha      D       13    NaN    NaN         NaN         NaN   
1783  Pichincha      D       13    NaN    NaN         NaN         NaN   
1784  Pichincha      D       13    NaN    NaN         NaN         NaN   
1785  Pichincha      D       13    NaN    NaN         NaN         NaN   
1786  Pichincha      D       13    NaN    NaN         NaN         NaN   

     transferred  
1782         NaN  
1783         NaN  
1784         NaN  
1785         NaN  
178

In [14]:
merged_df_2['type_y'].fillna("Work Day", inplace=True)
merged_df_2

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,False


In [15]:
# Cuenta la frecuencia de cada valor en la columna 'type'
value_counts = merged_df_2["locale"].value_counts()

# Cuenta los valores NaN en la columna 'type'
nan_count = merged_df_2["locale"].isna().sum()

# Agrega el recuento de NaN al resultado anterior
value_counts_with_nan = pd.concat([value_counts, pd.Series([nan_count], index=['NaN'])])

print(value_counts_with_nan)

National     261954
Local        208494
Regional      32076
NaN         2551824
dtype: int64


In [16]:
# Cuenta la frecuencia de cada valor en la columna 'type'
value_counts = merged_df_2["locale_name"].value_counts()

# Cuenta los valores NaN en la columna 'type'
nan_count = merged_df_2["locale_name"].isna().sum()

# Agrega el recuento de NaN al resultado anterior
value_counts_with_nan = pd.concat([value_counts, pd.Series([nan_count], index=['NaN'])])

print(value_counts_with_nan)

Ecuador                            261954
Guayaquil                           19602
Riobamba                            17820
Guaranda                            16038
Latacunga                           16038
Quito                               14256
Ambato                              14256
Cuenca                              10692
Manta                                8910
Esmeraldas                           8910
Cayambe                              8910
Santo Domingo                        8910
El Carmen                            8910
Machala                              8910
Imbabura                             8910
Puyo                                 8910
Libertad                             8910
Cotopaxi                             8910
Ibarra                               7128
Quevedo                              7128
Santo Domingo de los Tsachilas       7128
Santa Elena                          7128
Loja                                 7128
Salinas                           

In [17]:
# Filtra las filas donde 'category' tiene un valor NaN
nan_rows = merged_df_2[merged_df_2['locale_name'].isna()]

# Muestra los primeros ejemplos (por ejemplo, los primeros 5)
print(nan_rows.head())

        id        date  store_nbr      family   sales  onpromotion   city  \
1782  1782  2013-01-02          1  AUTOMOTIVE     2.0            0  Quito   
1783  1783  2013-01-02          1   BABY CARE     0.0            0  Quito   
1784  1784  2013-01-02          1      BEAUTY     2.0            0  Quito   
1785  1785  2013-01-02          1   BEVERAGES  1091.0            0  Quito   
1786  1786  2013-01-02          1       BOOKS     0.0            0  Quito   

          state type_x  cluster    type_y locale locale_name description  \
1782  Pichincha      D       13  Work Day    NaN         NaN         NaN   
1783  Pichincha      D       13  Work Day    NaN         NaN         NaN   
1784  Pichincha      D       13  Work Day    NaN         NaN         NaN   
1785  Pichincha      D       13  Work Day    NaN         NaN         NaN   
1786  Pichincha      D       13  Work Day    NaN         NaN         NaN   

     transferred  
1782         NaN  
1783         NaN  
1784         NaN  
1785

In [18]:
merged_df_2['locale'].fillna("Work Day", inplace=True)
merged_df_2['locale_name'].fillna("Work Day", inplace=True)
merged_df_2['description'].fillna("Work Day", inplace=True)
merged_df_2 = merged_df_2.drop("transferred", axis=1)
merged_df_2

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba


# Oil

In [19]:
oil_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
oil_data

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


In [20]:
merged_df_3 = pd.merge(merged_df_2, oil_data, on='date', how='left')
merged_df_3

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,NaN
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,NaN
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,NaN
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,NaN
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57


In [21]:
median_value = merged_df_3['dcoilwtico'].median()
merged_df_3['dcoilwtico'].fillna(median_value, inplace=True)
merged_df_3


,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
1,1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
2,2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
3,3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
4,4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054344,3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054345,3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054346,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57


In [22]:
# Cuenta los valores NaN en la columna 'type'
nan_count = merged_df_3["dcoilwtico"].isna().sum()

print(nan_count)

0


# Encode the data

Ahora queda categorizar los valores de cada columna para que todo quede en formato numerico para los algoritmos

In [23]:
merged_df_3 = merged_df_3.drop(["id"], axis=1)
merged_df_3

,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,dcoilwtico
0,2013-01-01,1,AUTOMOTIVE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
1,2013-01-01,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
2,2013-01-01,1,BEAUTY,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
3,2013-01-01,1,BEVERAGES,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
4,2013-01-01,1,BOOKS,0.000,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,53.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054344,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054345,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57
3054346,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,Holiday,Local,Riobamba,Fundacion de Riobamba,47.57


In [24]:
from sklearn.preprocessing import LabelEncoder

# Columnas a codificar
cols_to_encode = ['family', 'city', 'state', 'type_x', 'type_y', 'locale', 'locale_name', 'description']

# Aplicar LabelEncoder a cada columna
for col in cols_to_encode:
    le = LabelEncoder()
    merged_df_3[col] = le.fit_transform(merged_df_3[col])

merged_df_3


,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,dcoilwtico
0,2013-01-01,1,0,0.000,0,18,12,3,13,3,1,4,50,53.43
1,2013-01-01,1,1,0.000,0,18,12,3,13,3,1,4,50,53.43
2,2013-01-01,1,2,0.000,0,18,12,3,13,3,1,4,50,53.43
3,2013-01-01,1,3,0.000,0,18,12,3,13,3,1,4,50,53.43
4,2013-01-01,1,4,0.000,0,18,12,3,13,3,1,4,50,53.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054343,2017-08-15,9,28,438.133,0,18,12,1,6,3,0,19,28,47.57
3054344,2017-08-15,9,29,154.553,1,18,12,1,6,3,0,19,28,47.57
3054345,2017-08-15,9,30,2419.729,148,18,12,1,6,3,0,19,28,47.57
3054346,2017-08-15,9,31,121.000,8,18,12,1,6,3,0,19,28,47.57


# Dividir datos temporales

In [25]:
merged_df_3 = pd.DataFrame(merged_df_3)

# Ordenar el DataFrame por fecha
merged_df_3 = merged_df_3.sort_values(by="date")

# Establecer un punto de corte (por ejemplo, 80% para entrenamiento y 20% para prueba)
train_size = int(0.8 * len(merged_df_3))

# Dividir en conjuntos de entrenamiento y prueba
train = merged_df_3[:train_size]
test = merged_df_3[train_size:]

# Separar las características y las etiquetas
X_train = train.drop(["sales", "date"], axis=1)
y_train = train["sales"]

X_test = test.drop(["sales", "date"], axis=1)
y_test = test["sales"]

X_train, y_train, X_test, y_test

(         store_nbr  family  onpromotion  city  state  type_x  cluster  type_y  \
 0                1       0            0    18     12       3       13       3   
 1194            42       6            0     3      0       3        2       3   
 1193            42       5            0     3      0       3        2       3   
 1192            42       4            0     3      0       3        2       3   
 1191            42       3            0     3      0       3        2       3   
 ...            ...     ...          ...   ...    ...     ...      ...     ...   
 2444188         39      10           21     3      0       1        6       5   
 2444187         39       9            2     3      0       1        6       5   
 2444186         39       8            5     3      0       1        6       5   
 2444185         39       7           34     3      0       1        6       5   
 2444184         39       6            0     3      0       1        6       5   
 
          loca

In [26]:
len(X_train), len(y_train), len(X_test), len(y_test)

(2443478, 2443478, 610870, 610870)

# Análisis de Clases
Ahora debo separar por familia e iniciar el modelaje

In [27]:
families = merged_df_3['family'].unique()
family_data = {family: merged_df_3[merged_df_3['family'] == family] for family in families}

# Preprocesado para test data para las predicciones

In [28]:
def preprocess_data(test_data, stores_data, holidays_data, oil_data):
    """
    Preprocess the test data by:
    - Merging with stores, holiday, and oil data.
    - Handling missing values.
    
    Parameters:
    - test_data: The test dataset.
    - stores_data: The stores dataset with 'store_nbr' and other store-specific columns.
    - holidays_data: The holiday data with date information.
    - oil_data: The oil data with date and dcoilwtico information.
    
    Returns:
    - The preprocessed test data.
    """
    # Extract dates from the test dataset
    test_dates = test_data['date'].unique()
    
    # Filter the holidays and oil datasets for these dates
    holidays_data = holidays_data[holidays_data['date'].isin(test_dates)]
    oil_data = oil_data[oil_data['date'].isin(test_dates)]
    
    # Merge the test data with the stores, holidays, and oil data
    test_data = pd.merge(test_data, stores_data, on='store_nbr', how='left')
    test_data = pd.merge(test_data, holidays_data, on='date', how='left')
    test_data = pd.merge(test_data, oil_data, on='date', how='left')
    
    # Handle missing values
    test_data['type_y'].fillna("Work Day", inplace=True)
    test_data['locale'].fillna("Work Day", inplace=True)
    test_data['locale_name'].fillna("Work Day", inplace=True)
    test_data['description'].fillna("Work Day", inplace=True)
    
    median_value = test_data['dcoilwtico'].median()
    test_data['dcoilwtico'].fillna(median_value, inplace=True)
    
    return test_data

def encode_categorical_columns(data, encoders=None):
    """
    Encode several categorical columns using LabelEncoder.
    
    Parameters:
    - data: The dataset to be encoded.
    - encoders: A dictionary of pre-fitted LabelEncoders. If None, new encoders will be fitted.
    
    Returns:
    - The encoded data.
    - The encoders used for encoding.
    """
    cols_to_encode = ['family', 'city', 'state', 'type_x', 'type_y', 'locale', 'locale_name', 'description']
    
    if encoders is None:
        encoders = {}
    
    for col in cols_to_encode:
        if col in encoders:
            le = encoders[col]
        else:
            le = LabelEncoder()
            encoders[col] = le
            le.fit(data[col])
        data[col] = le.transform(data[col])
    
    return data, encoders


# Modelo por familia

In [29]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Ensure using TensorFlow 2.0
print(tf.__version__)

models = {}

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenamiento de modelos para todas las familias
for family in family_data.keys():
    # Specific checkpoint path for each family
    checkpoint_path = f"best_model_for_{family}.h5"
    model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)
    
    # Prepare the data
    data = family_data[family]
    X = data.drop(['sales', 'date'], axis=1)
    y = data['sales']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Build the model
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1)
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    # Train the model with callbacks
    model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), 
              callbacks=[early_stopping, model_checkpoint])
    
    # Load the best model weights from the checkpoint
    model.load_weights(checkpoint_path)
    
    # Store the trained model
    models[family] = model

    # Print the completion of the family's training
    print(f"Training for family '{family}' has finished.")

2.12.0
Epoch 1/50
2314/2314 [==============================] - 7s 3ms/step - loss: 30.2304 - val_loss: 27.3627
Epoch 2/50
2314/2314 [==============================] - 6s 2ms/step - loss: 25.9841 - val_loss: 23.8671
Epoch 3/50
2314/2314 [==============================] - 6s 3ms/step - loss: 24.0083 - val_loss: 21.7698
Epoch 4/50
2314/2314 [==============================] - 6s 2ms/step - loss: 22.8676 - val_loss: 20.9367
Epoch 5/50
2314/2314 [==============================] - 6s 2ms/step - loss: 22.1893 - val_loss: 20.7119
Epoch 6/50
2314/2314 [==============================] - 6s 2ms/step - loss: 21.8241 - val_loss: 21.1880
Epoch 7/50
2314/2314 [==============================] - 6s 2ms/step - loss: 21.5802 - val_loss: 19.8152
Epoch 8/50
2314/2314 [==============================] - 6s 3ms/step - loss: 21.2963 - val_loss: 20.2186
Epoch 9/50
2314/2314 [==============================] - 6s 3ms/step - loss: 21.1893 - val_loss: 19.8906
Epoch 10/50
2314/2314 [==============================] - 

# Predicciones finales para csv submit

In [30]:
# Predicciones para todas las familias
stores_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
holidays_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
oil_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")

test_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
preprocessed_test_data = preprocess_data(test_data, stores_data, holidays_data, oil_data)
preprocessed_test_data, _ = encode_categorical_columns(preprocessed_test_data)

submission_results = pd.DataFrame(columns=['id', 'sales'])

for family, model in models.items():
    # Filtrar el conjunto de datos de prueba para esta familia
    data = preprocessed_test_data[preprocessed_test_data['family'] == family]

    # Preparar las características de entrada para la predicción
    X_pred = data.drop(['id', 'date', 'transferred'], axis=1)

    # Hacer predicciones
    y_pred = model.predict(X_pred).flatten()

    # Añadir resultados al dataframe de envío
    family_results = pd.DataFrame({
        'id': data['id'],
        'sales': y_pred
    })
    submission_results = pd.concat([submission_results, family_results], ignore_index=True)

# Generar archivo de envío
submission_results.to_csv("submission_final.csv", index=False)

27/27 [==============================] - 0s 2ms/step


In [31]:
from IPython.display import FileLink
FileLink('submission_final.csv')

/kaggle/working/submission_final.csv

# --------------------------------------------

# --------------------------------------------

# Nuevo modelo más corto para verificar el flujo

In [32]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Ensure using TensorFlow 2.0
print(tf.__version__)

models = {}

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Specific checkpoint path for family 0
family = 0
checkpoint_path = f"best_model_for_{family}.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

# Prepare the data
data = family_data[family]
X = data.drop(['sales', 'date'], axis=1)
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model with callbacks
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), 
          callbacks=[early_stopping, model_checkpoint])

# Load the best model weights from the checkpoint
model.load_weights(checkpoint_path)

# Store the trained model
models[family] = model

# Print the completion of the training
print(f"Training for family '{family}' has finished.")


2.12.0
Epoch 1/50
2314/2314 [==============================] - 7s 3ms/step - loss: 29.3165 - val_loss: 25.8351
Epoch 2/50
2314/2314 [==============================] - 6s 3ms/step - loss: 24.7912 - val_loss: 22.6468
Epoch 3/50
2314/2314 [==============================] - 6s 2ms/step - loss: 23.0687 - val_loss: 22.7305
Epoch 4/50
2314/2314 [==============================] - 6s 3ms/step - loss: 22.3797 - val_loss: 20.7152
Epoch 5/50
2314/2314 [==============================] - 6s 3ms/step - loss: 21.9105 - val_loss: 20.6870
Epoch 6/50
2314/2314 [==============================] - 6s 3ms/step - loss: 21.5654 - val_loss: 20.4872
Epoch 7/50
2314/2314 [==============================] - 6s 3ms/step - loss: 21.2172 - val_loss: 19.6998
Epoch 8/50
2314/2314 [==============================] - 6s 3ms/step - loss: 21.1980 - val_loss: 20.8195
Epoch 9/50
2314/2314 [==============================] - 6s 3ms/step - loss: 20.9399 - val_loss: 19.8668
Epoch 10/50
2314/2314 [==============================] - 

In [33]:
# Evaluar el modelo en el conjunto de prueba
loss = model.evaluate(X_test, y_test)

print(f"Mean Squared Error en el conjunto de prueba: {loss}")

579/579 [==============================] - 1s 2ms/step - loss: 18.2482
Mean Squared Error en el conjunto de prueba: 18.248205184936523


# Mejorando el modelo: solo con una clase
Seleccionado el modelo lo incluiré en todas las clases (o familias)

In [34]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import BatchNormalization, Dropout

# Ensure using TensorFlow 2.0
print(tf.__version__)

models = {}

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Specific checkpoint path for family 0
family = 0
checkpoint_path = f"best_model_for_{family}.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

# Prepare the data
data = family_data[family]
X = data.drop(['sales', 'date'], axis=1)
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
specified_model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1)
])

specified_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')

# Train the model with callbacks
specified_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), 
          callbacks=[early_stopping, model_checkpoint])

# Load the best model weights from the checkpoint
specified_model.load_weights(checkpoint_path)

# Store the trained model
models[family] = specified_model

# Print the completion of the training
print(f"Training for family '{family}' has finished.")


2.12.0
Epoch 1/100
2314/2314 [==============================] - 7s 3ms/step - loss: 31.1178 - val_loss: 28.3540
Epoch 2/100
2314/2314 [==============================] - 6s 3ms/step - loss: 28.1561 - val_loss: 26.4273
Epoch 3/100
2314/2314 [==============================] - 6s 2ms/step - loss: 26.9447 - val_loss: 25.4386
Epoch 4/100
2314/2314 [==============================] - 6s 3ms/step - loss: 26.1759 - val_loss: 24.9155
Epoch 5/100
2314/2314 [==============================] - 6s 2ms/step - loss: 25.5972 - val_loss: 24.2835
Epoch 6/100
2314/2314 [==============================] - 6s 2ms/step - loss: 25.0721 - val_loss: 23.9403
Epoch 7/100
2314/2314 [==============================] - 6s 2ms/step - loss: 24.5992 - val_loss: 24.8396
Epoch 8/100
2314/2314 [==============================] - 6s 3ms/step - loss: 24.1284 - val_loss: 24.0323
Epoch 9/100
2314/2314 [==============================] - 6s 3ms/step - loss: 23.7415 - val_loss: 23.2728
Epoch 10/100
2314/2314 [========================

In [35]:
# Evaluar el modelo en el conjunto de prueba
loss = specified_model.evaluate(X_test, y_test)

print(f"Mean Squared Error en el conjunto de prueba: {loss}")

579/579 [==============================] - 1s 2ms/step - loss: 19.6395
Mean Squared Error en el conjunto de prueba: 19.63947105407715


In [36]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import BatchNormalization, Dropout

# Ensure using TensorFlow 2.0
print(tf.__version__)

models = {}

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Specific checkpoint path for family 0
family = 0
checkpoint_path = f"best_model_for_{family}.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

# Prepare the data
data = family_data[family]
X = data.drop(['sales', 'date'], axis=1)
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
model_2 = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(1)
])

model_2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model with callbacks
model_2.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), 
          callbacks=[early_stopping, model_checkpoint])

# Load the best model weights from the checkpoint
model_2.load_weights(checkpoint_path)

# Store the trained model
models[family] = model_2

# Print the completion of the training
print(f"Training for family '{family}' has finished.")

2.12.0
Epoch 1/50
2314/2314 [==============================] - 8s 3ms/step - loss: 32.2604 - val_loss: 26.7096
Epoch 2/50
2314/2314 [==============================] - 7s 3ms/step - loss: 27.2587 - val_loss: 23.2168
Epoch 3/50
2314/2314 [==============================] - 7s 3ms/step - loss: 25.5495 - val_loss: 21.8398
Epoch 4/50
2314/2314 [==============================] - 7s 3ms/step - loss: 24.5785 - val_loss: 21.5861
Epoch 5/50
2314/2314 [==============================] - 7s 3ms/step - loss: 24.0862 - val_loss: 20.9574
Epoch 6/50
2314/2314 [==============================] - 7s 3ms/step - loss: 23.7503 - val_loss: 20.7068
Epoch 7/50
2314/2314 [==============================] - 7s 3ms/step - loss: 23.4460 - val_loss: 21.1349
Epoch 8/50
2314/2314 [==============================] - 7s 3ms/step - loss: 22.9826 - val_loss: 20.8273
Epoch 9/50
2314/2314 [==============================] - 7s 3ms/step - loss: 22.9599 - val_loss: 22.7172
Epoch 10/50
2314/2314 [==============================] - 

In [37]:
# Evaluar el modelo en el conjunto de prueba
loss = model_2.evaluate(X_test, y_test)

print(f"Mean Squared Error en el conjunto de prueba: {loss}")

579/579 [==============================] - 1s 2ms/step - loss: 20.7068
Mean Squared Error en el conjunto de prueba: 20.706785202026367


In [38]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.layers import LeakyReLU

# Ensure using TensorFlow 2.0
print(tf.__version__)

models = {}

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Specific checkpoint path for family 0
family = 0
checkpoint_path = f"best_model_for_{family}.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

# Prepare the data
data = family_data[family]
X = data.drop(['sales', 'date'], axis=1)
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the improved model with slight modifications
model_3 = keras.Sequential([
    keras.layers.Dense(256, activation=LeakyReLU(alpha=0.01), input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    keras.layers.Dense(128, activation=LeakyReLU(alpha=0.01), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    keras.layers.Dense(64, activation=LeakyReLU(alpha=0.01), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    keras.layers.Dense(1)
])

model_3.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='mean_squared_error')

# Train the model with callbacks
model_3.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), 
          callbacks=[early_stopping, model_checkpoint])

# Load the best model weights from the checkpoint
model_3.load_weights(checkpoint_path)

# Store the trained model
models[family] = model_3

# Print the completion of the training
print(f"Training for family '{family}' has finished.")

2.12.0
Epoch 1/50
2314/2314 [==============================] - 13s 5ms/step - loss: 30.4166 - val_loss: 26.8145
Epoch 2/50
2314/2314 [==============================] - 11s 5ms/step - loss: 28.1130 - val_loss: 24.9984
Epoch 3/50
2314/2314 [==============================] - 10s 5ms/step - loss: 28.3183 - val_loss: 25.1261
Epoch 4/50
2314/2314 [==============================] - 11s 5ms/step - loss: 28.0005 - val_loss: 27.1105
Epoch 5/50
2314/2314 [==============================] - 11s 5ms/step - loss: 27.8131 - val_loss: 26.0889
Epoch 6/50
2314/2314 [==============================] - 12s 5ms/step - loss: 27.3996 - val_loss: 24.3805
Epoch 7/50
2314/2314 [==============================] - 12s 5ms/step - loss: 27.3876 - val_loss: 25.3858
Epoch 8/50
2314/2314 [==============================] - 11s 5ms/step - loss: 27.0998 - val_loss: 27.9973
Epoch 9/50
2314/2314 [==============================] - 11s 5ms/step - loss: 26.9472 - val_loss: 24.3582
Epoch 10/50
2314/2314 [=========================

In [39]:
# Evaluar el modelo en el conjunto de prueba
loss = model_3.evaluate(X_test, y_test)

print(f"Mean Squared Error en el conjunto de prueba: {loss}")

579/579 [==============================] - 1s 2ms/step - loss: 22.4909
Mean Squared Error en el conjunto de prueba: 22.490888595581055


In [40]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.layers import LeakyReLU

# Ensure using TensorFlow 2.0
print(tf.__version__)

models = {}

# Callbacks
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Specific checkpoint path for family 0
family = 0
checkpoint_path = f"best_model_for_{family}.h5"
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

# Prepare the data
data = family_data[family]
X = data.drop(['sales', 'date'], axis=1)
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a deeper model
model_4 = keras.Sequential([
    keras.layers.Dense(512, activation=LeakyReLU(alpha=0.01), input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    keras.layers.Dense(256, activation=LeakyReLU(alpha=0.01), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    keras.layers.Dense(128, activation=LeakyReLU(alpha=0.01), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    keras.layers.Dense(64, activation=LeakyReLU(alpha=0.01), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),

    keras.layers.Dense(32, activation=LeakyReLU(alpha=0.01), kernel_regularizer=keras.regularizers.l2(0.001)),
    BatchNormalization(),
    Dropout(0.3),
    
    keras.layers.Dense(1)
])

model_4.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='mean_squared_error')

# Train the model with callbacks
model_4.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), 
          callbacks=[early_stopping, model_checkpoint])

# Load the best model weights from the checkpoint
model_4.load_weights(checkpoint_path)

# Store the trained model
models[family] = model_4

# Print the completion of the training
print(f"Training for family '{family}' has finished.")

2.12.0
Epoch 1/50
2314/2314 [==============================] - 25s 9ms/step - loss: 32.1598 - val_loss: 28.3267
Epoch 2/50
2314/2314 [==============================] - 22s 9ms/step - loss: 30.1818 - val_loss: 27.6525
Epoch 3/50
2314/2314 [==============================] - 24s 10ms/step - loss: 30.1136 - val_loss: 26.4359
Epoch 4/50
2314/2314 [==============================] - 23s 10ms/step - loss: 29.7934 - val_loss: 26.8157
Epoch 5/50
2314/2314 [==============================] - 22s 10ms/step - loss: 29.6825 - val_loss: 25.9474
Epoch 6/50
2314/2314 [==============================] - 22s 10ms/step - loss: 29.3850 - val_loss: 31.1564
Epoch 7/50
2314/2314 [==============================] - 23s 10ms/step - loss: 29.0843 - val_loss: 25.9407
Epoch 8/50
2314/2314 [==============================] - 22s 10ms/step - loss: 28.9921 - val_loss: 26.3811
Epoch 9/50
2314/2314 [==============================] - 23s 10ms/step - loss: 28.5097 - val_loss: 25.5887
Epoch 10/50
2314/2314 [==================

In [41]:
# Evaluar el modelo en el conjunto de prueba
loss = model_4.evaluate(X_test, y_test)

print(f"Mean Squared Error en el conjunto de prueba: {loss}")

579/579 [==============================] - 2s 3ms/step - loss: 23.9790
Mean Squared Error en el conjunto de prueba: 23.979001998901367


## El modelo 1 (el más sencillo) es el mejor

In [42]:
# (Usar las funciones `preprocess_data` y `encode_categorical_columns` proporcionadas anteriormente)

# 1. Preprocesar los datos
stores_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
holidays_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
oil_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")

test_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
preprocessed_test_data = preprocess_data(test_data, stores_data, holidays_data, oil_data)

# 2. Codificar las columnas categóricas
preprocessed_test_data, _ = encode_categorical_columns(preprocessed_test_data)
# 3. Predicción
submission_results = pd.DataFrame(columns=['id', 'sales'])

# Filtrar el conjunto de datos de prueba para la familia 0
data = preprocessed_test_data[preprocessed_test_data['family'] == family]

# Preparar las características de entrada para la predicción
X_pred = data.drop(['id', 'date'], axis=1)
X_pred = X_pred.drop(['transferred'], axis=1)

# Hacer predicciones
y_pred = model.predict(X_pred).flatten()

# Añadir resultados al dataframe de envío
family_results = pd.DataFrame({
    'id': data['id'],
    'sales': y_pred
})
submission_results = pd.concat([submission_results, family_results], ignore_index=True)

# 4. Generar archivo de envío
submission_results.to_csv("submission.csv", index=False)

27/27 [==============================] - 0s 2ms/step
